# Erstellen neuer Features

Zum Erstellen der neuen Features werden zunächst die Verkaufsdaten (train.csv) und die Geschäftsdaten (store.csv) geladen und anschließend eine umfangreiche Datenvorverarbeitung und Feature-Engineering durchgeführt. Nachdem die Daten geladen und in ein Pandas DataFrame eingelesen wurden, wird die `Date`-Spalte in ein Datetime-Format konvertiert und verschiedene zeitbezogene Features wie Jahr, Monat, Tag, Wochentag und Kalenderwoche erstellt. Wochenenden, Feiertage und Schulferien werden als binäre Features kodiert, um diese speziellen Tage zu identifizieren. Anschließend werden die Verkaufsdaten (train) mit den Geschäftsdaten (`Store`) anhand der Store-ID zusammengeführt, um zusätzliche geschäftsspezifische Informationen hinzuzufügen. Lag Features, die die Verkaufszahlen der vorhergehenden Tage und Wochen darstellen, werden erstellt. Dabei werden die Verkaufszahlen eines Tages, einer Woche und eines Monats vorher berücksichtigt. Zusätzlich werden Rolling Features, die den gleitenden Durchschnitt und die Standardabweichung der Verkaufszahlen über bestimmte Zeiträume darstellen, erstellt. Store-spezifische Features wie die Dauer, seitdem ein Wettbewerber geöffnet hat (`competition_open_since`), und die Dauer, seitdem eine Promotion läuft (`promo2_since`), werden berechnet. Promotionsintervalle werden in binäre Features (`is_promo_month`) umgewandelt, um anzugeben, ob der aktuelle Monat Teil eines Promotionsintervalls ist. Weiterhin werden Dummy-Variablen für kategorische Features wie `StoreType`, `Assortment` und `StateHoliday` erstellt, um diese in das Modell zu integrieren. Nicht-numerische Spalten werden entfernt, um sicherzustellen, dass nur numerische Daten für die Modellierung verwendet werden. Fehlende Werte in den Daten werden mit Null aufgefüllt. Schließlich wird die Korrelation zwischen den neu erstellten Features und den vorhandenen Spalten berechnet und in einer Korrelationsmatrix dargestellt. Diese Matrix zeigt die Korrelation der neuen Features (`lag_1`, `lag_7`, `lag_30`, `rolling_mean_7`, `rolling_mean_30`, `rolling_std_7`, `rolling_std_30`, `competition_open_since`, `promo2_since`, `is_promo_month`) mit den bestehenden Features (`Sales`, `Customers`, `Open`, `Promo`, `Promo2`, `SchoolHoliday`, `CompetitionDistance`) an, um deren Beziehung und potenziellen Einfluss auf die Verkaufszahlen zu analysieren.

In [127]:
import pandas as pd
import numpy as np

In [128]:
file_store = "../data/store.csv"
file_train = "../data/train.csv"

In [129]:
store = pd.read_csv(file_store, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
train = pd.read_csv(file_train, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

In [130]:
# convert date and create temporal features
train['Date'] = pd.to_datetime(train['Date'])
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
#train['day_of_week'] = train['Date'].dt.dayofweek
train['week_of_year'] = train['Date'].dt.isocalendar().week

# Weekends, public holidays and school vacations are coded as binary features.
#train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)
#train['is_holiday'] = (train['StateHoliday'] != '0').astype(int)
#train['is_school_holiday'] = train['SchoolHoliday']

# Merge train dataset with store dataset using the store ID to add store-specific information
data_info = pd.merge(train, store, on='Store')

# Create lag features (sales of the previous days/weeks)
data_info['lag_1'] = data_info.groupby('Store')['Sales'].shift(1)
data_info['lag_7'] = data_info.groupby('Store')['Sales'].shift(7)
data_info['lag_30'] = data_info.groupby('Store')['Sales'].shift(30)

# Create rolling features (moving average and standard deviation)
data_info['rolling_mean_7'] = data_info.groupby('Store')['Sales'].shift(1).rolling(window=7).mean()
data_info['rolling_mean_30'] = data_info.groupby('Store')['Sales'].shift(1).rolling(window=30).mean()
data_info['rolling_std_7'] = data_info.groupby('Store')['Sales'].shift(1).rolling(window=7).std()
data_info['rolling_std_30'] = data_info.groupby('Store')['Sales'].shift(1).rolling(window=30).std()

# Create store-specific features (= duration since a competitor has opened)
data_info['competition_open_since'] = (
    (data_info['year'] - data_info['CompetitionOpenSinceYear']) * 12 +
    (data_info['month'] - data_info['CompetitionOpenSinceMonth'])
)

# Create promotion features (= duration since a promotion has been running)
data_info['promo2_since'] = (
    (data_info['year'] - data_info['Promo2SinceYear']) * 52 +
    (data_info['week_of_year'] - data_info['Promo2SinceWeek'])
)


promo_intervals = {'Jan,Apr,Jul,Oct': [1, 4, 7, 10],
                   'Feb,May,Aug,Nov': [2, 5, 8, 11],
                   'Mar,Jun,Sept,Dec': [3, 6, 9, 12]}
# Promotion intervals are converted into binary features (is_promo_month), see Hinweis                 
data_info['is_promo_month'] = data_info.apply(lambda row: 1 if row['month'] in promo_intervals.get(row['PromoInterval'], []) else 0, axis=1)

# Create dummy variables for categorical features
data_info = pd.get_dummies(data_info, columns=['StoreType', 'Assortment', 'StateHoliday'], drop_first=True)

# Remove non-numeric columns
non_numeric_columns = data_info.select_dtypes(include=['object']).columns
data_info = data_info.drop(columns=non_numeric_columns)

# Fill in missing values
data_info.fillna(0, inplace=True)

# Calculate correlation
correlation_matrix = data_info.corr()

# Show correlation of the new features with the existing columns
new_features = ['lag_1', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7', 'rolling_std_30', 'competition_open_since', 'promo2_since', 'is_promo_month']
existing_features = ['Sales', 'Customers', 'Open', 'Promo', 'Promo2', 'SchoolHoliday', 'CompetitionDistance']

correlation_new_vs_existing = correlation_matrix.loc[new_features, existing_features]

print(correlation_new_vs_existing)


KeyboardInterrupt: 

In [70]:
data_info['lag_1']

0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
            ...   
1017204     5097.0
1017205    10797.0
1017206     6218.0
1017207    20642.0
1017208     3697.0
Name: lag_1, Length: 1017209, dtype: float64

## Hinweis:
1. Nicht-numerische Spalten entfernen: Spalten, die Zeichenketten enthalten, werden vor der Berechnung der Korrelationen entfernt.
2. Fehlende Werte auffüllen: Fehlende Werte werden mit 0 aufgefüllt, um sicherzustellen, dass keine NaN-Werte die Korrelationsberechnung beeinträchtigen.

Dieser Ansatz stellt sicher, dass nur numerische Daten in die Berechnung der Korrelationen einfließen und vermeidet den ValueError.

# Analyse der neu erstellten Features

Um zu beurteilen, ob die neu erstellten Features für die Vorhersage der Verkäufe geeignet sind, eignet sich die Korrelationsmatrix. Ein neues Feature ist gut geeignet, wenn es eine hohe positive oder negative Korrelation mit den Verkäufen (`Sales`) aufweist. Eine hohe positive Korrelation bedeutet, dass das Feature einen starken positiven Zusammenhang mit den Verkaufszahlen hat, was es in der Regel nützlich für Vorhersagemodelle macht. Eine hohe negative Korrelation deutet ebenfalls auf eine starke Beziehung hin, jedoch in die entgegengesetzte Richtung, was ebenfalls hilfreich sein kann. Zusätzlich sind hohe Korrelationen mit anderen relevanten Features wie `Promo` nützlich, da sie auf mögliche indirekte Einflüsse auf die Verkäufe hinweisen können. Anhand der erstellten Korrelationsmatrix können neuen Features wie folgt bewertet werden:

- Das Feature `lag_1` zeigt eine mäßig positive Korrelation mit `Sales` (0.276397), was es zu einem mäßigen Prädiktor für Verkäufe macht. Für `SchoolHoliday` (0.033883) und `CompetitionDistance` (-0.018596) ist die Korrelation jedoch sehr schwach. Insgesamt kann `lag_1` nützlich sein, insbesondere für kurzfristige Vorhersagen.

- Das Feature `lag_7` weist eine hohe positive Korrelation mit `Sales` (0.662853) auf, was es zu einem starken Prädiktor für diese Variable macht. Die Korrelation mit `SchoolHoliday` (0.088015) und `CompetitionDistance` (-0.018304) ist jedoch sehr schwach. `lag_7` sollte dennoch aufgrund der hohen Korrelation mit `Sales` im Modell verwendet werden.

- Das Feature `lag_30` zeigt eine schwache positive Korrelation mit `Sales` (0.191832), sowie sehr schwache negative Korrelationen mit `SchoolHoliday` (-0.089403) und `CompetitionDistance` (-0.016816). Aufgrund der schwachen Korrelationen ist `lag_30` möglicherweise weniger nützlich.

- Die `rolling_mean_7`- und `rolling_mean_30`-Features zeigen insgesamt sehr schwache Korrelationen mit `Sales`, `Customers`, `SchoolHoliday` und `CompetitionDistance`, was darauf hinweist, dass sie schwache Prädiktoren sind.

- Die `rolling_std_7`- und `rolling_std_30`-Features zeigen ebenfalls sehr schwache Korrelationen mit den betrachteten Variablen, was ihre Nützlichkeit als Prädiktoren weiter einschränkt.

- Das Feature `competition_open_since` weist durchweg sehr schwache negative Korrelationen mit `Sales`, `SchoolHoliday` und `CompetitionDistance` auf, was darauf hindeutet, dass es ein sehr schwacher Prädiktor ist.

- Das Feature `promo2_since` zeigt ebenfalls sehr schwache negative Korrelationen mit allen betrachteten Variablen, was es zu einem weiteren schwachen Prädiktor macht.

- Das Feature `is_promo_month` zeigt sehr schwache negative Korrelationen mit `Sales`, sowie eine sehr schwache positive Korrelation mit `SchoolHoliday`, was seine Nützlichkeit als Prädiktor weiter einschränkt.

Auf Grundlage der Korrelationsmatrix sind insbesondere die Features `lag_7` und `lag_1` nützlich, da sie starke bzw. mäßige positive Korrelationen mit `Sales` aufweisen. Die anderen neuen Features zeigen sehr schwache Korrelationen mit `Sales` und anderen wichtigen Spalten, was darauf hinweist, dass sie weniger nützlich für die Vorhersage sind. Es kann jedoch sinnvoll sein, einige dieser Features weiterhin zu berücksichtigen und ihre Wirkung in einem tatsächlichen Modell zu testen, da die Korrelation allein nicht immer die volle Aussagekraft eines Features zeigt.

# Erstellen des Features Promo

Ein Problem in den vorliegenden Daten ist, dass die Spalten "Promo2SinceWeek", "Promo2SinceYear" und "PromoInterval" NaN-Werte enthalten, sollte ein Store nicht an einer Promotion-Aktion teilnehmen. Da jedoch nicht jedes Modell mit NaN-Werten umgehen kann, müssen diese mit sinnvollen Werten ersetzt werden. Um dieses Problem zu lösen, wird eine neue Spalte in der Trainingsdatei erstellt, die die Werte Null oder Eins enthält. Null bedeutet, dass an diesem Tag in dem Store keine Promotion stattfindet, während Eins anzeigt, dass eine Promotion stattfindet. Um dies zu erreichen, werden die Spalten `Promo2`, `Promo2SinceWeek`, `Promo2SinceYear` und `PromoInterval` aus der `store`-Datei zusammengeführt und in die Trainingsdatei übernommen. Hierfür wird mit Hilfe der `Promo2`-Spalte überprüft ob ein Store an einer Aktion teilnimmt. Sollte dies der Fall sein, das heißt eine Eins ist eingetragen, so kann mit Hilfe der Spalten `Promo2SinceWeek`, `Promo2SinceYear` und `PromoInterval` jeder Tag berechnet werden an dem der Store an der Aktion teilgenommen hat. Für diese Tage wird in der `promo`-Spalte der Trainingsdatei eine Eins eingetragen. Für alle anderen Tage, das heißt Tage an denen keine Promotion in Stores stattfindet, wird eine Null eingetragen. Somit sind keine NaN-Werte mehr vorhanden, da diese mit sinnvollen Werten ersetzt wurden. Für jeden Tag wird daher, basierend auf den Informationen aus den `store`-Daten, berechnet, ob der Store an einer Promotion teilnimmt oder nicht. Die beschriebene Vorgehensweise ist möglich, da die `Promo2`-Spalte immer Werte enthält (Null oder Eins). Aus diesen Werten kann abgeleitet werden, dass NaN-Werte in den anderen Spalten entstehen, da der Store nicht an der Aktion teilnimmt und somit keine Werte in diese Spalten eingetragen werden können. Für die NaN-Zeilen können daher in der neu erstellten Promotionsspalte der Trainingsdatei eine 0 eingetragen.

In [131]:
# Convert date and create temporal features
train['Date'] = pd.to_datetime(train['Date'])
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['day_of_week'] = train['Date'].dt.dayofweek
train['week_of_year'] = train['Date'].dt.isocalendar().week
train['day_of_year'] = train['Date'].dt.dayofyear

# Encode weekends, public holidays and school vacations as binary features
train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)
train['is_holiday'] = (train['StateHoliday'] != '0').astype(int)
train['is_school_holiday'] = train['SchoolHoliday']

# Merge train data record with store data record using the store ID
data = pd.merge(train, store, on='Store')

# Fill NaN values in the promotion columns with 0
data['Promo2SinceWeek'].fillna(0, inplace=True)
data['Promo2SinceYear'].fillna(0, inplace=True)
data['PromoInterval'].fillna('', inplace=True)

# Create promotion column
def is_promo(row):
    if row['Promo2'] == 0:
        return 0
    if row['Promo2'] == 1:
        promo_start_year = int(row['Promo2SinceYear'])
        promo_start_week = int(row['Promo2SinceWeek'])
        current_year = int(row['year'])
        current_week = int(row['week_of_year'])
        
        # Calculation of whether the current date is in a promotion interval
        if current_year > promo_start_year or (current_year == promo_start_year and current_week >= promo_start_week):
            promo_intervals = {
                'Jan,Apr,Jul,Oct': [1, 4, 7, 10],
                'Feb,May,Aug,Nov': [2, 5, 8, 11],
                'Mar,Jun,Sept,Dec': [3, 6, 9, 12]
            }
            intervals = promo_intervals.get(row['PromoInterval'], [])
            if row['month'] in intervals:
                return 1
    return 0

data['promo2'] = data.apply(is_promo, axis=1)

# Remove unnecessary columns
data.drop(columns=['Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], inplace=True)

# Show result
print(data.head(5))

/var/folders/7l/z15ws_pd5x5_76bbfygfn4ch0000gn/T/ipykernel_57479/3820776584.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Promo2SinceWeek'].fillna(0, inplace=True)
/var/folders/7l/z15ws_pd5x5_76bbfygfn4ch0000gn/T/ipykernel_57479/3820776584.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2015-07-31   4822        559     1      1            0   

   SchoolHoliday  year  ...  day_of_year  is_weekend  is_holiday  \
0              1  2015  ...          212           0           0   
1              1  2015  ...          212           0           0   
2              1  2015  ...          212           0           0   
3              1  2015  ...          212           0           0   
4              1  2015  ...          212           0           0   

   is_school_holiday  StoreType  Assortment  CompetitionDistance  \
0                  1          c           a           

In [132]:
# Create Fourier features based on day of the year
data['fourier_sin_365'] = np.sin(2 * np.pi * data['day_of_year'] / 365)
data['fourier_cos_365'] = np.cos(2 * np.pi * data['day_of_year'] / 365)

In [133]:
# Filter holidays (where StateHoliday is not '0')
holidays = data[data['StateHoliday'] != '0'].copy()

In [135]:
# Calculate 'days_since_last_holiday'
data['days_since_last_holiday'] = data['Date'].apply(lambda x: (x - holidays[holidays['Date'] <= x]['Date'].max()).days)

In [137]:
# Calculate 'days_until_next_holiday'
data['days_until_next_holiday'] = data['Date'].apply(lambda x: (holidays[holidays['Date'] >= x]['Date'].min() - x).days)

In [117]:
# Step 2: Calculate 'days_until_next_holiday' for each date
def calculate_days_until_next_holiday(date):
    next_holiday_date = holidays[holidays['Date'] >= date]['Date'].min()
    if pd.isna(next_holiday_date):
        return np.nan
    return (next_holiday_date - date).days

data['days_until_next_holiday'] = data['Date'].apply(calculate_days_until_next_holiday)

In [138]:
# Step 3: Handle NaN values by setting them to the next known holiday (Oct 3, 2025)
next_holiday = pd.Timestamp('2015-10-03')

# Identify rows where 'days_until_next_holiday' is NaN
nan_mask = data['days_until_next_holiday'].isna()

# Correctly calculate the days until the next holiday for NaN rows
data.loc[nan_mask, 'days_until_next_holiday'] = (next_holiday - data.loc[nan_mask, 'Date']).dt.days

In [139]:
data['days_until_next_holiday'].describe()

count    1.017209e+06
mean     3.304999e+01
std      3.044408e+01
min      0.000000e+00
25%      8.000000e+00
50%      2.300000e+01
75%      5.500000e+01
max      1.200000e+02
Name: days_until_next_holiday, dtype: float64

# Auswahl vorhandener Merkmale

Die Auswahl weiterer Features für das Vorhersagemodell basiert auf ihrer logischen Relevanz und der Analyse ihrer Korrelation mit den Verkaufszahlen. 

- Der Wochentag (`DayOfWeek`) hat oft einen erheblichen Einfluss auf die Verkäufe, da die Kaufgewohnheiten der Kunden je nach Wochentag variieren können. Beispielsweise gibt es mehr Einkäufe an Wochenenden. Daher wird der Wochentag in Liste der verwendbaren Merkmale aufgenommen. 

- Das Merkmal `Customers` (Kundenanzahl) hingegen ist weniger geeignet. Diese Entscheidung basiert auf der Abwägung der Auswirkungen auf die Modellgenauigkeit und die Komplexität des Vorhersageprozesses. Die Einbeziehung dieses Features würde bedeuten, dass auch die Anzahl der Kunden für zukünftige Zeitpunkte vorhersagt werden müsste. Dies fügt eine zusätzliche Schicht von Komplexität hinzu, da nun zwei miteinander verknüpfte Vorhersagen getroffen werden müssten: die Anzahl der Kunden und darauf basierend die Verkaufszahlen. Die Vorhersage der Kundenanzahl würde zusätzliche Modelle oder Algorithmen erfordern, die ebenfalls eine gewisse Fehlerquote aufweisen. Diese Fehler würden sich in der Verkaufsprognose widerspiegeln und könnten die Genauigkeit des Verkaufsprognosemodells verringern. Durch den Ausschluss des `Customers`-Features wird eine potenzielle Fehlerquelle reduziert und der Fokus liegt auf direkt beobachtbaren und prognostizierbaren Merkmalen. Der bewusste Ausschluss dieses Features hilft dabei, die Vorhersagegenauigkeit zu maximieren und die Komplexität des Modells zu minimieren, was zu einem effizienteren und zuverlässigeren Vorhersagemodell führt.

- Ob ein Geschäft geöffnet oder geschlossen ist (`Open`), hat einen direkten Einfluss auf die Verkäufe. Geschäfte, die geschlossen sind, haben keine Verkäufe. Die Korrelationsmatrix zeigt erwartungsgemäß eine hohe Korrelation mit `Sales`, da Verkäufe nur an geöffneten Tagen stattfinden können. 

- Promotionen (`Promo`) beeinflussen die Verkaufszahlen erheblich, indem sie mehr Kunden anziehen und den Umsatz steigern. Die Korrelationsmatrix weist darauf hin, dass Promotionen zu höheren Verkaufszahlen führen, da eine positive Korrelation mit `Sales` besteht. 

- Feiertage (`StateHoliday`) beeinflussen das Einkaufsverhalten der Kunden, da an staatlichen Feiertagen Geschäfte geschlossen sein oder weniger Kunden haben könnten. Die Korrelation variiert je nach Art des Feiertags, zeigt aber signifikante Zusammenhänge. 

- Unterschiedliche Store-Typen (`StoreType`) können unterschiedliche Verkaufsmuster haben. Größere Geschäfte oder spezielle Geschäftsmodelle könnten höhere Verkäufe generieren. Die Korrelationsmatrix zeigt, dass die Korrelation je nach Typ variiert, aber signifikante Korrelationen mit anderen wichtigen Features wie `Promo` bestehen.

Die Wahl der zusätzlichen Features basiert auf ihrer logischen Relevanz und bisherigen Korrelationsanalysen. Features wie `DayOfWeek`, `Open`, `Promo`, `StateHoliday` und `StoreType` sind stark mit den Verkaufszahlen verbunden und bieten wertvolle Informationen zur Verbesserung der Modellgenauigkeit. Eine detaillierte Untersuchung der Korrelationen dieser Features mit `Sales` bestätigt ihre Bedeutung und rechtfertigt ihre Einbeziehung in das Vorhersagemodell.

In [140]:
# Create lag features (sales of the previous days/weeks)
data = data.sort_values(by=['Store', 'Date'])
data['lag_1'] = data.groupby('Store')['Sales'].shift(1)
data['lag_7'] = data.groupby('Store')['Sales'].shift(7)

data['lag_1'] = data.groupby(['Store', 'DayOfWeek'])['lag_1'].transform(lambda x: x.fillna(x.median()))
data['lag_7'] = data.groupby(['Store', 'DayOfWeek'])['lag_7'].transform(lambda x: x.fillna(x.median()))

data.drop(columns=['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Customers', 'SchoolHoliday'], inplace=True)

# Saving the cleansed data
data.to_csv('../data/cleaned_train.csv', index=False)

In [141]:
data[(data['month']==6) & (data['year']== 2015)]

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,year,month,day,...,is_school_holiday,StoreType,Assortment,promo2,fourier_sin_365,fourier_cos_365,days_since_last_holiday,days_until_next_holiday,lag_1,lag_7
66900,1,1,2015-06-01,5774,1,1,0,2015,6,1,...,0,c,a,0,0.501242,-0.865307,7,3.0,0.0,0.0
65785,1,2,2015-06-02,5450,1,1,0,2015,6,2,...,0,c,a,0,0.486273,-0.873807,8,2.0,5774.0,4211.0
64670,1,3,2015-06-03,5809,1,1,0,2015,6,3,...,0,c,a,0,0.471160,-0.882048,9,1.0,5450.0,4083.0
63555,1,4,2015-06-04,0,0,1,a,2015,6,4,...,0,c,a,0,0.455907,-0.890028,0,0.0,5809.0,4111.0
62440,1,5,2015-06-05,5384,1,1,0,2015,6,5,...,0,c,a,0,0.440519,-0.897743,1,120.0,0.0,4656.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40139,1115,5,2015-06-26,5549,1,0,0,2015,6,26,...,0,d,c,1,0.094537,-0.995521,22,99.0,5015.0,8291.0
39024,1115,6,2015-06-27,6676,1,0,0,2015,6,27,...,0,d,c,1,0.077386,-0.997001,23,98.0,5549.0,7824.0
37909,1115,7,2015-06-28,0,0,0,0,2015,6,28,...,0,d,c,1,0.060213,-0.998186,24,97.0,6676.0,0.0
36794,1115,1,2015-06-29,11006,1,1,0,2015,6,29,...,0,d,c,1,0.043022,-0.999074,25,96.0,0.0,5096.0


In [125]:
data[(data['StateHoliday'] != '0') & 
                     (data['year'] == 2014) & 
                     (data['month'] == 10)]

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,year,month,day,...,is_school_holiday,StoreType,Assortment,promo2,fourier_sin_365,fourier_cos_365,days_since_last_holiday,days_until_next_holiday,lag_1,lag_7
319595,1,5,2014-10-03,0,0,1,a,2014,10,3,...,0,c,a,0,-0.999250,0.038722,0,0.0,5400.0,3518.0
319596,2,5,2014-10-03,2689,1,1,a,2014,10,3,...,0,a,a,1,-0.999250,0.038722,0,0.0,4845.0,3583.0
293416,2,5,2014-10-31,0,0,0,a,2014,10,31,...,0,a,a,1,-0.867456,0.497513,0,0.0,5394.0,5096.0
319597,3,5,2014-10-03,0,0,1,a,2014,10,3,...,0,a,a,1,-0.999250,0.038722,0,0.0,8565.0,4853.0
319598,4,5,2014-10-03,0,0,1,a,2014,10,3,...,0,c,c,0,-0.999250,0.038722,0,0.0,12297.0,6948.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320525,1111,5,2014-10-03,0,0,1,a,2014,10,3,...,0,a,a,1,-0.999250,0.038722,0,0.0,5641.0,4719.0
320526,1112,5,2014-10-03,0,0,1,a,2014,10,3,...,0,c,c,0,-0.999250,0.038722,0,0.0,11863.0,6943.0
320527,1113,5,2014-10-03,0,0,1,a,2014,10,3,...,0,a,c,0,-0.999250,0.038722,0,0.0,8834.0,5668.0
320528,1114,5,2014-10-03,0,0,1,a,2014,10,3,...,0,a,c,0,-0.999250,0.038722,0,0.0,23557.0,17800.0


# Externe Features

Die Wetterbedingungen können einen erheblichen Einfluss auf das Kaufverhalten von Kunden und somit auf den Umsatz der Rossmann-Stores haben. Diese Beziehung ist in vielen Branchen gut dokumentiert, da das Wetter mit dazu beiträgt, ob und wo ein Kunde einkauft. Schlechtes Wetter wie Regen und Schnee können Menschen davon abhalten, ihre Häuser zu verlassen, was zu einem Rückgang der Kundenfrequenz und damit einhergehend einem Einbruch des Umsatzes zur folge haben kann. Andererseits können angenehme Wetterbedingungen die Menschen dazu ermutigen, mehr Zeit im Freien und in Einkaufszentren zu verbringen, was den Umsatz steigern kann. Ein weiteres Argument, dafür dass das Wetter das Kaufverhalten der Kunden beeinflussen kann, ist die Art der gekauften Produkte. Verbraucher kaufen an sonnigen und heißen Tagen viel mehr Erfrischungsgetränke und Eiscreme, wohingegen an kalten Tage die Nachfrage nach warmen Getränken steigt. Auch saisonale Veränderungen wie Ferienzeiten, die oft mit spezifischen Wetterbedingungen verbunden sind, können das Kaufverhalten und die Umsatzmuster stark beeinflussen. Angesichts dieser potenziellen Einflüsse ist es sinnvoll, Wetterbedingungen als externes Feature in Verkaufsprognosemodelle zu integrieren. Durch die Berücksichtigung von Wetterdaten wie Temperatur, Niederschlag, Sonnenscheindauer und Bewölkung kann die Genauigkeit der Vorhersagemodelle verbessert werden. Dies ermöglicht eine präzisere Planung und Steuerung von Lagerbeständen, Werbemaßnahmen und Personalressourcen.